In [11]:
import json
import glob
import csv

In [12]:
# ----------------------------------reading annotation files
def read_ann_file(file):
    f = open(file,"r",encoding="utf8")
    ann_Text = f.read()
    return ann_Text

In [13]:
# ------------------------------------reading labels  files
file ="data/labels.tsv"
labels = open(file,"r",encoding="utf8").read()

def get_conf_bias(file):
#     print("name recieved is : ",file)
    for item in labels.split("\n"):
        if file in item :
            confirmation_bias = item.strip().split()
#             print(confirmation_bias[1])
            if confirmation_bias[1] == "negative":
                return("false")
            elif confirmation_bias[1] =="positive":
                return("true")

In [14]:
#-------------------------------------getting paragraph text and anotation status
def get_Paragraphs(essay_no):
    with open("data/data-tokenized.tsv") as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
        for row in reader:
            if row['ESSAY'] == str(essay_no):
                dict_para['text'] = row['TEXT']
                if row['ANNOTATION'] == 'insufficient':
                      dict_para['sufficient'] = False
                elif row['ANNOTATION'] == '' :
                      dict_para['sufficient'] = True
            unified_file.append(dict_para)

In [15]:
# -----------------------reading text files
def get_text(file):
    essay_Text =open(file,"r",encoding="utf8").read()
#     print(text)
    return essay_Text

    

In [16]:
unified_file = []
ann_list = []
id_dict ={'id': 0 , "text" : ""}
# major_Claim_dict = {"major_claim" : [ "span" : 0 , "text": ""]}
# confirmation_dict = {"confirmation_bias" : True }
counter = 1
# dict_para = {'text':"",'sufficient' : True }

for file in glob.glob("data/brat-project-final/essay*.ann"):
    ann_text = read_ann_file(file)
    ann_file_name = file
    txt_file_name = ann_file_name.replace("ann","txt")
    id_dict['id'] = counter
    id_dict['text'] = get_text(txt_file_name)
    unified_file.append(id_dict)
#     for item in ann_text.split("\n"):
        
# # ---------------------------- Collecting Major claim
#         if "MajorClaim" in item :
#             ann_list = item.strip()
#             ann_list_words = ann_list.split()
#             MajorClaim=' '.join(ann_list_words[4:])
#             Span = ann_list_words[2:4]
# #             id_dict['major_claim'] = MajorClaim
# #             id_dict['span'] = Span
        
# #----------------------------- Collecting Claims 

#         if "Claim" in item and "MajorClaim" not in item :
#             ann_claim_list = item.strip()
#             ann_claim_words = ann_claim_list.split()
# #             print(ann_claim_words)
#             Claim=' '.join(ann_claim_words[4:])
# #             print("claim is : ",Claim)
#             Span = ann_claim_words[2:4]
# #             print("Span is ::",Span)
# #     print(id_dict)


# #---------------------------Collecting premises
#         if "Premise" in item :
#             ann_premise_list = item.strip()
#             ann_premise_words = ann_premise_list.split()
# #             print(ann_premise_words)
#             Premise=' '.join(ann_premise_words[4:])
# #             print("premise is : ",Premise)
#             Span = ann_premise_words[2:4]
# #             print("Span of premise is ::",Span)

# # -----------getting the confirmation bias. 
# #         print(file)
#     conf_file_name=file.replace("data/brat-project-final\essay","essay").replace(".ann","")
# #     print(conf_file_name)
#     confirmation_bias = get_conf_bias(conf_file_name)
# #     id_dict['confirmation_bias'] = confirmation_bias


#  ------------------collecting paragraphs 

#     get_Paragraphs(counter)
    counter = counter+1
    with open("just_id.json", "w") as outfile: 
#         json.dump(unified_file, outfile)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
with open("just_id.json","r") as f:
    users = json.load(f)

In [8]:
print(users[0:100])

[{'id': 402, 'text': "Children should studying hard or playing sport? Both can improve their future\n\nSome people believe that studying hard is essential part for children, otherwise others people think that playing sport is waste of time. In my point of view, both of studying hard and playing sports are part of life to children. \nOn the other hand, studying hard will give children a better future. Studying hard gives children with a plenty of knowledge and skills for their future. Naturally, children will be growing up and looking for their job. For instance, children who always studying hard and getting high achievement, it will be easy to pass qualification for work and they are more likely to find great work at multinational company. If they working at multinational company as the result is they can earn higher salary. The message is when you always studying harder than others you will get best results.\nAt the same time, playing sports will give good effects on children. There a